# EDA Notebook

This data comes from IBM Business Analytics and it is comprised of 4 Databases with different amount of data from each customer. The goal of this notebook is to compile all the data and clean it so it is ready for the modeling steps. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from sklearn.preprocessing import StandardScaler



pd.set_option('display.max_columns', 100)

In [2]:
#loading dataframes
df1=pd.read_excel("Telco_customer_churn.xlsx")
df2=pd.read_excel("Telco_customer_churn_status.xlsx")
df3=pd.read_excel("Telco_customer_churn_services.xlsx")
df4=pd.read_excel("Telco_customer_churn_demographics.xlsx")

In [3]:
#Mergin dataframes into one
df1_2=pd.merge(df1, df2, left_on='CustomerID', right_on='Customer ID', how='left').drop('Customer ID', axis=1)
df3_4=pd.merge(df3, df4, left_on='Customer ID', right_on='Customer ID', how='left')
customer_df=pd.merge(df1_2, df3_4, left_on='CustomerID', right_on='Customer ID', how='left').drop('Customer ID', axis=1)

In [4]:
#Dropping duplicates from DF and renaming the columns 
customer_df=customer_df.T.drop_duplicates().T
customer_df.columns=[column.lower().strip("_x").strip("_y").replace(" ","_")for column in customer_df.columns] 

In [5]:
def remove_dups(lists):
    """
    Takes in a list of duplicated columns with the same name and removes the duplicated column and creates a
    new column with a new name and values assigned.
    Once that is complete it drops column from dataframe.
    """
    for i in range(len(lists)):
        customer_df["tel_"+lists[i]]=customer_df[lists[i]].iloc[:,1:2]
        customer_df.drop(lists[i],axis=1,inplace=True)

In [6]:
remove_dup=["internet_service","online_securit","online_backup","multiple_lines","streaming_tv","payment_method","total_charges","contract","streaming_movies","churn_reason"]

In [7]:
#Removing duplicates that were not caught by the previous function.
remove_dups(remove_dup)

In [8]:
#Dropping Redundant columns
customer_df.drop(["lat_long","customerid","churn_label","count","countr","state","quarter",],axis=1,inplace=True)

In [9]:
#Cleaning missing values from "total_charge" column
hold=[]
for x in customer_df.index:
    if type(customer_df.tel_total_charges[x])== str:
        hold.append(0)
    else:
        hold.append(customer_df.tel_total_charges[x])
customer_df.tel_total_chargs=hold

<ipython-input-9-3d09f234fd45>:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  customer_df.tel_total_chargs=hold


In [10]:
#Binning age 
customer_df.age=pd.cut(x=customer_df['age'], bins=[10,20,30,40, 50,60,70,80], labels=["10","20","30","40", "50","60","70"])

In [11]:
#Filling NA values for churn reason
customer_df.tel_churn_reason=customer_df.tel_churn_reason.fillna("No reason given")

Filling the column of Churn Category, by comparing if the customer churned or not. If the customer didnt churn, Fill it with "Not Churn" to add the category that the customer did not leave, therefore filling the missing valuees. If the customer left, then fill it with "No reason" since we dont have a reason of why they left.

In [12]:
conditions=[
    (customer_df["churn_categor"].isna())& (customer_df["churn_value"]==0),            
    (customer_df["churn_categor"].isna())& (customer_df["churn_value"]==1),
]
choises=[
    "Not Churned",
    "No Reason",

]
customer_df.churn_categor=np.select(conditions,choises,default=customer_df.churn_categor)

In [13]:
 def to_binary(column):
    """
    Takes in a column and turns its value from a Yes or No to a binary. 
    """
    conditions=[
        (customer_df[column].str.lower()=='yes'),            
        (customer_df[column].str.lower()=='no'),
    ]
    choises=[
        1,
        0,

    ]
    customer_df[column]=np.select(conditions,choises,default=0)

In [14]:
def reason_cleanup(column):
    """
    Takes in the tel_churn_reason column and creates its own main categories as to why a customer churned. 
    
    """    
    
    reasons={
        "Competition Offers":["Competitor had better devices","Competitor made better offer","Competitor offered more data","Competitor offered higher download speeds"],
        "Customer Satisfaction":["Attitude of support person","Attitude of service provider","Poor expertise of online support","Poor expertise of phone support"],
        "Pricing":["Price too high","Lack of affordable download/upload speed"],
        "Charges and Fees":["Long distance charges","Extra data charges"],
        "Product and Services":["Product dissatisfaction","Network reliability","Service dissatisfaction","Limited range of services","Lack of self-service on Website"],
        "External Factors":["Moved","Deceased"],
        "Unkown":["Don't know"],
        "Did Not Churn":["No reason given"]
    }
    reason=[]
    for i in customer_df[column].index:
        for key in reasons.keys():
            if customer_df[column][i]in reasons[key]:
                reason.append(key)
    return reason

In [15]:
# Re categorizing the churn reason for better interpretation.
customer_df["tel_churn_reason"]=reason_cleanup("tel_churn_reason")

In [16]:
#converting columns into binary
col_to_bin=["senior_citizen","partner","dependents","phone_service","device_protection","tech_support","paperless_billing","referred_a_friend","device_protection_plan","premium_tech_support","streaming_music","unlimited_data","under_30","tel_internet_service","tel_online_securit","tel_online_backup","tel_multiple_lines","tel_streaming_tv","tel_streaming_movies"]
for x in col_to_bin:
    to_binary(x)

In [17]:
#The clean Dataframe is saved to be used for Visualizations in the Visualizations notebook.
customer_df.to_csv("clean_customer_df.csv")

####  Preparing Dataframe for modeling. 

This part the categorical variables are made into dummies making sure to drop the first value. In the cleaning process, some features are not in the right format for modeling. I change the type of the DF to Float so all the data is able to be processed. 

In [18]:
customer_df.drop(["tel_churn_reason","churn_categor","customer_status"],axis=1, inplace=True)

In [19]:
customer_df=pd.get_dummies(customer_df,columns=["cit","zip_code","latitude","longitude","gender","offer","internet_type","age","tel_payment_method","tel_contract"],drop_first=True)

In [20]:
customer_df=customer_df.astype("float")

In [21]:
customer_df.to_csv("modeling_customer_df.csv")

# Vanilla Model

For a Vanilla model, I want to explore a linear regression model. Since a Linear regression is the most basic and simple model to run without any parameter tuning, it can help me get an idea of where my data is at.

In [50]:
#importing necessary libraries.
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

client = Client(processes=False)  

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57561 instead
  warnings.warn(


In [51]:
y=customer_df["churn_value"]
X=customer_df[customer_df.columns[customer_df.columns!="churn_value"]]

In [52]:
# keeping a random state to keep concistency on each model
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2020, test_size=0.2)

In [53]:
scaler=StandardScaler()
X_test=scaler.fit_transform(X_test)
X_train=scaler.transform(X_train)

In [54]:
# Fitting our model
lr=LogisticRegression(solver="sag" ,max_iter=1000, random_state=40, verbose=1, n_jobs=-1)

lr.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


max_iter reached after 306 seconds


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


convergence after 823 epochs took 252 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.2min finished


LogisticRegression(max_iter=1000, n_jobs=-1, random_state=40, solver='sag',
                   verbose=1)

In [55]:
print("Train Accuracy:",lr.score(X_train, y_train))
print("Test Accuracy:",lr.score(X_test, y_test))

Train Accuracy: 0.9973375931842385
Test Accuracy: 0.9198012775017743


In [56]:
y_pred_lr = lr.predict(X_test)

In [57]:
print('Logistic Regression Accuracy: {}'.format(metrics.accuracy_score(y_test, y_pred_lr)))
print('Logistic Regression F1: {}'.format(metrics.f1_score(y_test,y_pred_lr)))

Logistic Regression Accuracy: 0.9198012775017743
Logistic Regression F1: 0.855683269476373


This model has performed very well for a vanilla model. This model will be explored further in the Modeling notebook.